In [2]:
import pandas as pd
from tqdm.auto import tqdm
import json, ijson
from pathlib import Path

p_narr = Path('/data/gunsbrother/prjs/ltvu/tasks/Ego4D/annotations/narration.json')
p_narr_red = Path('/data/gunsbrother/prjs/ltvu/tasks/Ego4D/annotations/all_narrations_redacted.json')

In [22]:
f = p_narr.open('rb')
it = ijson.parse(f)
video_ids = []
for prefix, event, value in it:
    if prefix and '.' not in prefix and event == 'start_map' and value is None:
        video_ids.append(prefix)
f.close()
len(video_ids), video_ids[:5]

(9611,
 ['77cc4654-4eec-44c6-af05-dbdf71f9a401',
  '3e08beb0-9108-4e77-b2ae-80f91ceac474',
  '6e389032-9216-4da0-8548-2f5087882589',
  '8a3c4a95-0492-41cf-802d-4198ab843582',
  'c3f5972e-9919-496c-a01a-75ffa5c7bcff'])

In [23]:
fr = p_narr_red.open('rb')
it = ijson.parse(fr)
video_ids_red = []
for prefix, event, value in it:
    if prefix == 'videos' and event == 'map_key' and '-' in value:
        video_ids_red.append(value)
fr.close()
len(video_ids_red), video_ids_red[:5]

(8922,
 ['cba13243-5ad3-4360-87df-6286867c473a',
  '81fb9f8f-be18-4833-a899-4fbcef82a634',
  '666e9020-446c-4cdc-9468-42af9188ce80',
  'd1c7b7ec-506e-4c02-b008-8b5aaaf442d7',
  '4bbc5b0a-5eb4-4973-a57a-45feceee9f39'])

In [25]:
len(set(video_ids_red) - set(video_ids)), len(set(video_ids) - set(video_ids_red))

(0, 689)

In [6]:
f = p_narr.open('rb')
next(ijson.items(f, video_ids[0]))

{'narration_pass_1': {'narrations': [{'timestamp_sec': Decimal('0.0'),
    'timestamp_frame': 0,
    '_unmapped_timestamp_sec': Decimal('0.0'),
    'narration_text': '#C C interacts with a woman X',
    'annotation_uid': '920182f7-5385-488b-99f9-caf8f0d9fe6b'},
   {'timestamp_sec': Decimal('4.53806'),
    'timestamp_frame': 136,
    '_unmapped_timestamp_sec': Decimal('4.53806'),
    'narration_text': '#C C walks into the kitchen',
    'annotation_uid': '920182f7-5385-488b-99f9-caf8f0d9fe6b'},
   {'timestamp_sec': Decimal('12.92098'),
    'timestamp_frame': 388,
    '_unmapped_timestamp_sec': Decimal('12.92098'),
    'narration_text': '#C C opens a shelf',
    'annotation_uid': '920182f7-5385-488b-99f9-caf8f0d9fe6b'},
   {'timestamp_sec': Decimal('15.10264'),
    'timestamp_frame': 453,
    '_unmapped_timestamp_sec': Decimal('15.10264'),
    'narration_text': '#C C brings out a basket from the shelf',
    'annotation_uid': '920182f7-5385-488b-99f9-caf8f0d9fe6b'},
   {'timestamp_sec': De

In [10]:
next(ijson.items(p_narr_red.open('rb'), 'videos.cba13243-5ad3-4360-87df-6286867c473a'))

{'narrations': [{'time': Decimal('0.0420572'),
   'text': '#C C converses with man X',
   '_clip_time_start': 0,
   '_clip_time_end': 0,
   '_end_time': Decimal('0.0420572'),
   'annotator': '1695609517163352',
   '_project_id': '3111279962304712',
   '_annotation_uid': '8a908b99-0e31-4427-aa93-c4a2509bbb88'},
  {'time': Decimal('0.7925471999999999'),
   'text': '#C C looks at the green vegetables ',
   '_clip_time_start': Decimal('0.75049'),
   '_clip_time_end': Decimal('0.75049'),
   '_end_time': Decimal('0.7925471999999999'),
   'annotator': '1695609517163352',
   '_project_id': '3111279962304712',
   '_annotation_uid': '8a908b99-0e31-4427-aa93-c4a2509bbb88'},
  {'time': Decimal('11.898337199999999'),
   'text': '#C C cuts vegetables on a chair with her hands.',
   '_clip_time_start': Decimal('11.85628'),
   '_clip_time_end': Decimal('11.85628'),
   '_end_time': Decimal('11.898337199999999'),
   'annotator': '4288081301262926',
   '_project_id': '347545199711956',
   '_annotation_ui

In [26]:
f, fr = p_narr.open('rb'), p_narr_red.open('rb')
p_df_dir = Path('/data/gunsbrother/prjs/ltvu/ours/data/ego4d')
p_df_dir.mkdir(exist_ok=True, parents=True)
p_csv = p_df_dir / 'narrations.csv'
p_pkl = p_df_dir / 'narrations.pkl'
f_csv, f_pkl = p_csv.open('w'), p_pkl.open('wb')
for idx, video_id in tqdm(enumerate(video_ids_red), total=len(video_ids_red)):
    video_narr_json = next(ijson.items(fr, f'videos.{video_id}'))
    fr.seek(0)
    df = pd.DataFrame.from_records(
        video_narr_json['narrations'],
        columns=['_annotation_uid', 'annotator', 'time', 'text', '_project_id', ])
    df['time'] = df['time'].astype('f4')
    df['text'] = df['text'].str.replace('\n', ' ').str.strip().str.replace('  ', ' ')
    df.insert(0, 'video_id', video_id)
    # f.seek(0)
    # if video_narr_json['status'] == 'redacted':
    #     print(video_id)
    # narr1 = video_narr_json['narration_pass_1']['narrations']
    # narr2 = video_narr_json['narration_pass_2']['narrations']
    # df = pd.DataFrame.from_records(narr1 + narr2, exclude=['_unmapped_timestamp_sec'])
    # df.insert(0, 'video_id', video_id)
    # df.insert(1, 'narration_pass', [1] * len(narr1) + [2] * len(narr2))
    # cols = list(df.columns)
    # df = df[cols[:2] + cols[-1:] + cols[3:-1]]
    df.to_csv(f_csv, header=idx==0, index=False, float_format='%.5f')
    df.to_pickle(f_pkl)
f_csv.close(), f_pkl.close()
df.head()

  0%|          | 0/8922 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [31]:
f = p_narr.open('rb')
it = ijson.parse(f)
for i, comp in enumerate(it):
    print(comp)
    if not flag and comp[0].startswith(video_ids[0]): flag = 1
    if flag and not comp[0].startswith(video_ids[0]):
        break

('', 'start_map', None)
('', 'map_key', '77cc4654-4eec-44c6-af05-dbdf71f9a401')
('77cc4654-4eec-44c6-af05-dbdf71f9a401', 'start_map', None)
('77cc4654-4eec-44c6-af05-dbdf71f9a401', 'map_key', 'narration_pass_1')
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1', 'start_map', None)
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1', 'map_key', 'narrations')
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1.narrations', 'start_array', None)
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1.narrations.item', 'start_map', None)
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1.narrations.item', 'map_key', 'timestamp_sec')
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1.narrations.item.timestamp_sec', 'number', Decimal('0.0'))
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1.narrations.item', 'map_key', 'timestamp_frame')
('77cc4654-4eec-44c6-af05-dbdf71f9a401.narration_pass_1.narrations.item.timestamp_frame', 'number', 0)
('77cc4654-4eec-44c6-a